In [2]:
%run ../mercadona/scraper.py

In [27]:
import pandas as pd
import sys
import os
import time
import random

In [14]:
# Lets get all categories and subcategories
zip = '08012'
categories = get_categories(zip)

list_of_dicts = []
for i in categories:
    subcat = get_subcategories(zip,i)
    for s in subcat:
        row = {}
        row["category"] = i
        row["subcategory"] = s
        list_of_dicts.append(row)

cats_and_subcats = pd.DataFrame(list_of_dicts)
cats_and_subcats

,category,subcategory
0,"Aceite, especias y salsas","Aceite, vinagre y sal"
1,"Aceite, especias y salsas",Especias
2,"Aceite, especias y salsas","Mayonesa, ketchup y mostaza"
3,"Aceite, especias y salsas",Otras salsas
4,Agua y refrescos,Agua
...,...,...
147,Postres y yogures,Yogures y postres infantiles
148,Zumos,Fruta variada
149,Zumos,Melocotón y piña
150,Zumos,Naranja


In [15]:
mercadona_structure = cats_and_subcats

In [16]:
missing = pd.read_csv('../missing.csv', index_col=0)
missing

,0
0,Especias
1,Isotónico y energético
2,Patatas fritas y snacks
3,Pasta y fideos
4,Galletas
5,Embutido curado
6,Jamón serrano
7,Queso untable y fresco
8,Verdura
9,Coloración cabello


In [17]:
mask = cats_and_subcats["subcategory"].isin(missing["0"])
mask

0      False
1       True
2      False
3      False
4      False
       ...  
147    False
148    False
149    False
150    False
151    False
Name: subcategory, Length: 152, dtype: bool

In [35]:
missing_subcats_2 = cats_and_subcats[mask]
missing_subcats_2.reset_index(inplace=True,drop=True)
missing_subcats_2

,category,subcategory
0,"Aceite, especias y salsas",Especias
1,Agua y refrescos,Isotónico y energético
2,Aperitivos,Patatas fritas y snacks
3,"Arroz, legumbres y pasta",Pasta y fideos
4,Cereales y galletas,Galletas
5,Charcutería y quesos,Embutido curado
6,Charcutería y quesos,Jamón serrano
7,Charcutería y quesos,Queso untable y fresco
8,Congelados,Verdura
9,Cuidado del cabello,Coloración cabello


In [20]:
# Now lets get what we scraped previously
productos_mercadona = pd.read_csv('../productos_mercadona.csv')
productos_mercadona.head()

,product,product_type,product_volume,product_price_per_unit,product_price,product_unit,product_category,product_subcategory,product_url,product_code,collected_timestamp
0,"Aceite de oliva 0,4º Hacendado",Garrafa,5 L,"| 4,726 €/L",23.63,/ud.,"Aceite, especias y salsas >","Aceite, vinagre y sal",https://tienda.mercadona.es/product/4241/aceit...,4241,2023-03-09 20:47:58.491387
1,"Aceite de oliva 0,4º Hacendado",Botella,1 L,"| 4,77 €/L",4.77,/ud.,"Aceite, especias y salsas >","Aceite, vinagre y sal",https://tienda.mercadona.es/product/4240/aceit...,4240,2023-03-09 20:47:59.149612
2,Aceite de oliva virgen extra Hacendado,Garrafa,3 L,"| 5,584 €/L",16.75,/ud.,"Aceite, especias y salsas >","Aceite, vinagre y sal",https://tienda.mercadona.es/product/4717/aceit...,4717,2023-03-09 20:47:59.687822
3,Aceite de oliva virgen extra Hacendado,Botella,1 L,"| 5,63 €/L",5.63,/ud.,"Aceite, especias y salsas >","Aceite, vinagre y sal",https://tienda.mercadona.es/product/4740/aceit...,4740,2023-03-09 20:48:00.227091
4,Aceite de oliva virgen extra Hacendado Gran Se...,Botella,750 ml,"| 6,814 €/L",5.11,/ud.,"Aceite, especias y salsas >","Aceite, vinagre y sal",https://tienda.mercadona.es/product/4706/aceit...,4706,2023-03-09 20:48:00.786828


In [40]:
# Lets get the missing categories


cod_postal = '08012'

start_time=time.time()

product_info = pd.DataFrame({})
error_count = 0
missing_subcats = []
for cat, row in missing_subcats_2.iterrows():
    print(f'\rGetting products for the "{row["subcategory"]}" subcategory in the "{row["category"]}" category...                                                ')
    sys.stdout.flush()
    try:
        products, product_count =  get_product_info(zip, row["category"], row["subcategory"], headless=False)
        product_info = pd.concat([product_info,products], ignore_index=True)
        random_time = random.randint((0.2*60*1000), (1*60*1000)) /1000
        print(f"\n---------------\nTime:{round((time.time()-start_time)/60,2)}\nFinished '{row['subcategory']}' subcateogry succesfully. \n{product_count} products registered.\nWaiting {round(random_time/60,2)} minutes so that we don't get caught...\nCurrent size of data captured: {product_info.shape}\n---------------\n")
        time.sleep(random_time)

    except:
        print(f'\n\nTime: {round((time.time()-start_time)/60,2)}')
        random_time = random.randint((((0.2*60)+(error_count*5))*1000), (((1*60)+(error_count*5))*1000)) /1000
        error_count +=1
        print(f'!!! An error occurred in subcategory "{row["subcategory"]}". Retrying in {round(random_time/60,2)} minutes...\n')
        time.sleep(random_time)
        try:
            products, product_count =  get_product_info(zip, row["category"], row["subcategory"], headless=False)
            product_info = pd.concat([product_info,products], ignore_index=True)
            random_time = random.randint((0.2*60*1000), (1*60*1000)) /1000
            print(f"\n---------------\nTime:{round((time.time()-start_time)/60,2)}\nFinished '{row['subcategory']}' subcateogry succesfully. \n{product_count} products registered.\nWaiting {round(random_time/60,2)} minutes so that we don't get caught...\nCurrent size of data captured: {product_info.shape}\n---------------\n")
            time.sleep(random_time)
        except:
            print(f'\n\nTime: {round((time.time()-start_time)/60,2)}')
            error_count +=1
            missing_subcats.append(row["subcategory"])
            print(f'!!! An error occurred in subcategory "{row["subcategory"]}". Moving on to the next category, appending this to the list of missing subcategories...\n')

Getting products for the "Especias" subcategory in the "Aceite, especias y salsas" category...                                                
Scraping "48: Preparado para ..." product...                                                                              

Time: 0.69
!!! An error occurred in subcategory "Especias". Retrying in 0.69 minutes...

Scraping "23: Ajo granulado H..." product...                                                                              

Time: 1.8
!!! An error occurred in subcategory "Especias". Moving on to the next category, appending this to the list of missing subcategories...

Getting products for the "Isotónico y energético" subcategory in the "Agua y refrescos" category...                                                
Scraping "25: Bebida energéti..." product...                                                                              

Time: 2.19
!!! An error occurred in subcategory "Isotónico y energético". Retrying in 1.04 minutes...

KeyboardInterrupt: 

In [3]:
cod_postal = os.getenv("cod_postal")

start_time=time.time()
print(f"\rGetting categories...                                                      ", end='')
sys.stdout.flush()
categories = get_categories(cod_postal)

product_info = pd.DataFrame({})
error_count = 0
missing_subcats = []

for i in categories:
    print(f'\rGetting subcategories for the "{i}" category...                                                      ', end='')
    sys.stdout.flush()
    subcategories = get_subcategories(cod_postal, i)
    for x in subcategories:
        print(f'\rGetting products for the "{x}" subcategory in the "{i}" category...                                                ')
        sys.stdout.flush()
        
        retries = 4
        while retries > 0:
            try:
                products, product_count =  get_product_info(cod_postal, i, x)
                product_info = pd.concat([product_info,products], ignore_index=True)
                random_time = random.randint((0.2*60*1000), (0.5*60*1000)) /1000
                print(f"\n---------------\nTime:{round((time.time()-start_time)/60,2)}\nFinished '{x}' subcateogry succesfully. \n{product_count} products registered.\nWaiting {round(random_time/60,2)} minutes so that we don't get caught...\nCurrent size of data captured: {product_info.shape}\n---------------\n")
                time.sleep(random_time)
                break

            except:
                print(f'\n\nTime: {round((time.time()-start_time)/60,2)}')
                random_time = random.randint((((0.2*60)+(error_count*10))*1000), (((0.5*60)+(error_count*10))*1000)) /1000
                error_count +=1
                retries -=1
                print(f'!!! An error occurred in subcategory "{x}". Retrying in {round(random_time/60,2)} minutes...\n')
                time.sleep(random_time)
            

product_info

Getting products for the "Aceite, vinagre y sal" subcategory in the "Aceite, especias y salsas" category...                                                
Scraping "39: Sal rosa del Hi..." product...                                                                              
---------------
Time:0.67
Finished 'Aceite, vinagre y sal' subcateogry succesfully. 
39 products registered.
Waiting 0.33 minutes so that we don't get caught...
Current size of data captured: (39, 11)
---------------

Getting products for the "Especias" subcategory in the "Aceite, especias y salsas" category...                                                
Scraping "17: Pimentón picant..." product...                                                                              

Time: 1.28
!!! An error occurred in subcategory "Especias". Retrying in 0.5 minutes...

Scraping "5: Albahaca Hacend..." product...                                                                              

Time: 1.95
!!! An error o

In [1]:
product_info

NameError: name 'product_info' is not defined

In [8]:
%run ../mercadona/scraper.py

In [10]:
product_info, mising_subcategories = mercadona_full_scraper('08012',retry=4, wait_min=0.01, wait_max=0.01, e_wait_min=0.01, e_wait_max=0.01, headless=False)

Getting products for the "Aceite, vinagre y sal" subcategory in the "Aceite, especias y salsas" category...                                                


Time: 0.26
!!! An error occurred in subcategory "Aceite, vinagre y sal". Retrying in 0.01 minutes...

Scraping "1: Aceite de oliva..." product...                                                                              

Time: 0.41
!!! An error occurred in subcategory "Aceite, vinagre y sal". Retrying in 0.18 minutes...

Scraping "4: Aceite de oliva..." product...                                                                              

Time: 0.73
!!! An error occurred in subcategory "Aceite, vinagre y sal". Retrying in 0.34 minutes...

Scraping "1: Aceite de oliva..." product...                                                                              

Time: 1.2
!!! An error occurred in subcategory "Aceite, vinagre y sal". Adding it to the list of missing subcategories...

Getting products for the "Especias" subcateg

KeyboardInterrupt: 

In [4]:
mising_subcategories


""
